<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0120tokenizer_summary_ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- filename: 2022_0119tokenizer_summary_ja.ipynb
- original: 
- file location: https://github.com/ShinAsakawa/ShinAsakawa.github.io/tree/master/2022notebooks
- date: 2022_0120

In [ ]:
import platform
isColab = platform.system() == 'Linux'
if isColab:
    # Transformers installation
    !pip install transformers datasets > /dev/null 2>&1 
    # To install from source instead of the last release, comment the command above and uncomment the following one.
    #!pip install git+https://github.com/huggingface/transformers.git

# トークナイザの要約 <!--# Summary of the tokenizers-->

このページでは，トークン化について詳しく見ていきます。
<!-- On this page, we will have a closer look at tokenization. -->

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/VFp38yj8h3A?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

[前処理のチュートリアル](https://huggingface.co/docs/transformers/master/en/preprocessing) で見たように，テキストのトークン化とは，テキストを単語やサブワードに分割し，それをルックアップテーブルで ID に変換することです。
単語やサブワードを ID に変換するのは簡単なので，このまとめでは，テキストを単語やサブワードに分割すること (すなわちテキストのトークン化) に焦点を当てます。
具体的には Transformer で使用されている 3 つの主要なタイプのトークン化器について見ていきます。
[Byte-Pair Encoding (BPE)](#byte-pair-encoding)，[WordPiece](#wordpiece)，[SentencePiece](#sentencepiece) です。
そして，どのモデルでどのタイプのトークナイザーが使われているか例を示します。
<!-- As we saw in [the preprocessing tutorial](https://huggingface.co/docs/transformers/master/en/preprocessing), tokenizing a text is splitting it into words or subwords, which then are converted to ids through a look-up table. 
Converting words or subwords to ids is straightforward, so in this summary, we will focus on splitting a text into words or subwords (i.e. tokenizing a text).
More specifically, we will look at the three main types of tokenizers used in 🤗 Transformers: [Byte-Pair Encoding (BPE)](#byte-pair-encoding), [WordPiece](#wordpiece), and [SentencePiece](#sentencepiece), and show examples of which tokenizer type is used by which model. -->

なお，各モデルのページでは，関連するトークナイザのドキュメントを見ることで，事前学習したモデルでどのトークナイザータイプが使われているかを知ることができます。
例えば [BertTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/bert#transformers.BertTokenizer) を見ると，モデルが [WordPiece](#wordpiece) を使用していることがわかります。
<!-- Note that on each model page, you can look at the documentation of the associated tokenizer to know which tokenizer type was used by the pretrained model. 
For instance, if we look at [BertTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/bert#transformers.BertTokenizer), we can see that the model uses [WordPiece](#wordpiece). -->


## はじめに <!--## Introduction-->

文章を小さな塊に分割することは，見た目以上に難しい作業であり，その方法も複数あります。
例えば 「トランスフォーマー大好きでしょ🤗？ わかってるよ。」という文を見てみましょう。
<!-- Splitting a text into smaller chunks is a task that is harder than it looks, and there are multiple ways of doing so.
For instance, let's look at the sentence `"Don't you love 🤗 Transformers? We sure do."` -->

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/nhJxYji1aho?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

このテキストをトークン化する簡単な方法は，スペースで分割することで，次のようになります。
<!-- A simple way of tokenizing this text is to split it by spaces, which would give: -->

```
["トランスフォーマー", "大好き", "でしょ", "?", "分かって", "る", "よ", "。"]
```


```
["Don't", "you", "love", "🤗", "Transformers?", "We", "sure", "do."]
```

これは賢明な第一歩ですが，トークン `"Transformer?"` と `"do."` を見ると，句読点が単語 `"Transformer"` と `"do"` に付けられていますが，これは最適ではありません。
句読点を考慮することで，モデルが単語とそれに続く可能性のあるすべての句読点記号の異なる表現を学習する必要がなくなり，モデルが学習しなければならない表現の数が爆発的に増えることになります。
句読点を考慮して，例示したテキストをトークン化すると，次のようになります。
<!-- This is a sensible first step, but if we look at the tokens `"Transformers?"` and `"do."`, we notice that the punctuation is attached to the words `"Transformer"` and `"do"`, which is suboptimal. 
We should take the punctuation into account so that a model does not have to learn a different representation of a word and every possible punctuation symbol that could follow it, which would explode the number of representations the model has to learn. 
Taking punctuation into account, tokenizing our exemplary text would give: -->

```
["Don", "'", "t", "you", "love", "🤗", "Transformers", "?", "We", "sure", "do", "."]
```

良いですね。
しかし，トークン化が `"Don't"` という単語をどのように処理したかが気になります。
`"Don't”` は `"do not”` の略ですから `["Do", "n't”]` としてトークン化するのが良いでしょう。
各モデルが独自のトークナイザー・タイプを持っている理由の 1 つは，ここからが複雑になってくるからです。
テキストのトークン化に適用するルールによって，同じテキストでも異なるトークン化された出力が生成されます。
事前に学習されたモデルは，学習データのトークン化に使用されたのと同じルールでトークン化された入力を与えられた場合にのみ，適切に動作します。
<!-- Better. 
However, it is disadvantageous, how the tokenization dealt with the word `"Don't"`. `"Don't"` stands for `"do not"`, so it would be better tokenized as `["Do", "n't"]`. 
This is where things start getting complicated, and part of the reason each model has its own tokenizer type. 
Depending on the rules we apply for tokenizing a text, a different tokenized output is generated for the same text. 
A pretrained model only performs properly if you feed it an input that was tokenized with the same rules that were used to tokenize its training data. -->

[spaCy](https://spacy.io/) と [Moses](http://www.statmt.org/moses/?n=Development.GetStarted) は 2 つの有名なルールベースのトークナイザーです。
これらを私たちの例に適用すると *spaCy* と *Moses* は次のように出力されます。
<!-- [spaCy](https://spacy.io/) and [Moses](http://www.statmt.org/moses/?n=Development.GetStarted) are two popular rule-based tokenizers. 
Applying them on our example, *spaCy* and *Moses* would output something like: -->

```
["Do", "n't", "you", "love", "🤗", "Transformers", "?", "We", "sure", "do", "."]
```

ここでは，スペースと句読点によるトークン化と，ルールベースのトークン化が使用されていることがわかります。 
スペースと句読点のトークン化とルールベースのトークン化は，どちらも単語のトークン化の例で，大まかには文を単語に分割することです。
これはテキストを小さな塊に分割する最も直感的な方法ですが，このトークン化方法は大量のテキストコーパスでは問題を引き起こす可能性があります。
この場合，スペースと句読点のトークン化は，通常，非常に大きな語彙 (使用されるすべてのユニークな単語とトークンのセット) を生成します。
*例えば*  [Transformer XL](https://huggingface.co/docs/transformers/master/en/model_doc/transformerxl) では，スペースと句読点によるトークン化を使用しているため，267,735 の語彙サイズになっています。
<!-- As can be seen space and punctuation tokenization, as well as rule-based tokenization, is used here. 
Space and punctuation tokenization and rule-based tokenization are both examples of word tokenization, which is loosely defined as splitting sentences into words. 
While it's the most intuitive way to split texts into smaller chunks, this tokenization method can lead to problems for massive text corpora. 
In this case, space and punctuation tokenization usually generates a very big vocabulary (the set of all unique words and tokens used). *E.g.*, [Transformer XL](https://huggingface.co/docs/transformers/master/en/model_doc/transformerxl) uses space and punctuation tokenization, resulting in a vocabulary size of 267,735! -->


このように語彙数が多いと，モデルの入力層と出力層に巨大な埋め込み行列が必要となり，メモリと時間の両方が増大してしまいます。
一般的に，変換モデルが 50,000 以上の語彙サイズを持つことは，特に 1 つの言語でのみ事前学習されている場合はほとんどありません。
<!-- Such a big vocabulary size forces the model to have an enormous embedding matrix as the input and output layer, which causes both an increased memory and time complexity. 
In general, transformers models rarely have a vocabulary size greater than 50,000, especially if they are pretrained only on a single language. -->

単純なスペースや句読点のトークン化では満足できないのであれば，単純に文字をトークン化すればいいのではないでしょうか。
<!-- So if simple space and punctuation tokenization is unsatisfactory, why not simply tokenize on characters? -->

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/ssLq_EK2jLE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

文字のトークン化は非常にシンプルで，メモリと時間の複雑さを大幅に軽減することができますが，モデルが意味のある入力表現を学習することは非常に難しくなります。
*例えば* 文字 `"t"` に対して意味のある文脈に依存しない表現を学習することは，単語 `"today"` に対して文脈に依存しない表現を学習することよりもはるかに困難です。
そのため，文字のトークン化はしばしば成績低下を伴います。
そこで，両方の利点を活かすために，トランスフォーマモデルでは，**サブワード** トークン化と呼ばれる，単語レベルと文字レベルのトークン化のハイブリッドを使用します。
<!-- While character tokenization is very simple and would greatly reduce memory and time complexity it makes it much harder for the model to learn meaningful input representations. 
*E.g.* learning a meaningful context-independent representation for the letter `"t"` is much harder than learning a context-independent representation for the word `"today"`. 
Therefore, character tokenization is often accompanied by a loss of performance. 
So to get the best of both worlds, transformers models use a hybrid between word-level and character-level tokenization called **subword** tokenization. -->


### サブワードトークン化 <!--### Subword tokenization-->

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/zHvTiHr506c?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

サブワード (部分単語) 型トークン化アルゴリズムは、頻繁に使われる単語はより小さな サブワード (部分単語) に分割すべきではないが，稀な単語 (低頻度語) は意味のあるサブワード (部分) に分解すべきであるという原則に基づいています。
例えば `"annouraingly”` は珍しい単語 (低頻度語) とみなされ，`"annouraingly”` と `"ly”` に分解される可能性があります。
この場合 `"annouraingly”` と `"ly”` の両方が独立したサブワード (部分単語) としてより頻繁に出現すると同時に，`"annouraingly”` の意味は `"annouraingly”` と `"ly”` の合成意味によって維持されることになります。
これはトルコ語のような膠着語では特に有効で，サブワード (部分単語) をつなぎ合わせることで （ほぼ) 任意に長い複合語を形成することができます。
<!-- Subword tokenization algorithms rely on the principle that frequently used words should not be split into smaller subwords, but rare words should be decomposed into meaningful subwords. 
For instance `"annoyingly"` might be considered a rare word and could be decomposed into `"annoying"` and `"ly"`. 
Both `"annoying"` and `"ly"` as stand-alone subwords would appear more frequently while at the same time the meaning of `"annoyingly"` is kept by the composite meaning of `"annoying"` and `"ly"`. 
This is especially useful in agglutinative languages such as Turkish, where you can form (almost) arbitrarily long complex words by stringing together subwords. -->

サブワード (部分単語) 型トークン化により，モデルは適度な語彙数を持ちながら，意味のある文脈に依存しない表現を学習することができます。
さらに，サブワード (部分単語) 型トークン化により，モデルは初めて目にする単語を既知のサブワード (部分単語) に分解して処理することができます。
例えば [BertTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/bert#transformers.BertTokenizer) は  `"I have a new GPU!”`  を以下のようにトークン化します。
<!-- Subword tokenization allows the model to have a reasonable vocabulary size while being able to learn meaningful context-independent representations. 
In addition, subword tokenization enables the model to process words it has never seen before, by decomposing them into known subwords. 
For instance, the [BertTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/bert#transformers.BertTokenizer) tokenizes `"I have a new GPU!"` as follows:
-->

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.tokenize("I have a new GPU!")

['i', 'have', 'a', 'new', 'gp', '##u', '!']

大文字小文字を区別しないモデルを考えているため，まず文が小文字にしました。
単語  `["i", "have", "a", "new"]`  はトークン化器の語彙に存在します。
ですが， 単語 `"gpu"` は存在しないことが分かります。
その結果，トークン化器は  `"gpu"`  を既知のサブワードに分割します。
`[“gp”]`  と `[ "##u"]`に分割されます。
ここで，`"##”` は，トークンの残りの部分をスペースなしで前のトークンにくっつけることを意味します (復号化やトークン化の反転のため)。
<!-- Because we are considering the uncased model, the sentence was lowercased first. We can see that the words `["i", "have", "a", "new"]` are present in the tokenizer's vocabulary, but the word `"gpu"` is not. 
Consequently, the tokenizer splits `"gpu"` into known subwords: `["gp" and "##u"]`. `"##"` means that the rest of the token should be attached to the previous one, without space (for decoding or reversal of the tokenization). -->


別例として，[XLNetTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/xlnet#transformers.XLNetTokenizer) は，先に例示したテキストを次のようにトークン化します。
<!-- As another example, [XLNetTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/xlnet#transformers.XLNetTokenizer) tokenizes our previously exemplary text as follows: -->

In [ ]:
from transformers import XLNetTokenizer

tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

['▁Don',
 "'",
 't',
 '▁you',
 '▁love',
 '▁',
 '🤗',
 '▁',
 'Transform',
 'ers',
 '?',
 '▁We',
 '▁sure',
 '▁do',
 '.']

この `”▁”` の意味については [SentencePiece](#sentencepiece) を見るときにまた説明します。
見ての通り，珍しい単語 (低頻度語)  `"Transformers"` はより頻繁に使われるサブワード (部分単語) `"Transform"` と `"ers”` に分割されていることがわかります。
<!-- We'll get back to the meaning of those `"▁"` when we look at [SentencePiece](#sentencepiece). 
As one can see, the rare word `"Transformers"` has been split into the more frequent subwords `"Transform"` and `"ers"`.-->

次に，異なるサブワード (部分単語)  トークン化アルゴリズムがどのように機能するかを見てみましょう。
これらのトークン化アルゴリズムはすべて，対応するモデルが学習されるコーパスで通常行われる，何らかの学習に依存していることに注意してください。
<!-- Let's now look at how the different subword tokenization algorithms work. 
Note that all of those tokenization algorithms rely on some form of training which is usually done on the corpus the corresponding model will be trained on.
-->

<a id='byte-pair-encoding'></a>

## Byte-Pair Encoding (BPE)

Byte-Pair Encoding (BPE) は [Neural Machine Translation of Rare Words with Subword Units (Sennrich et al., 2015)](https://arxiv.org/abs/1508.07909) で紹介されたものです。
BPE は，学習データを単語に分割する 事前トークン化器に依存します。
事前トークン化器は，例えば [GPT-2](https://huggingface.co/docs/transformers/master/en/model_doc/gpt2),  [Roberta](https://huggingface.co/docs/transformers/master/en/model_doc/roberta)  のように，空白トークン化器と同じくらい単純なものでもあります。
より高度な事前トークン化としては，規則に基づくトークン化，たとえば  [XLM](https://huggingface.co/docs/transformers/master/en/model_doc/xlm) があります。
[FlauBERT](https://huggingface.co/docs/transformers/master/en/model_doc/flaubert) では、ほとんどの言語で Moses トークン化器を使用し，[GPT](https://huggingface.co/docs/transformers/master/en/model_doc/gpt) では Spacy と ftfy を使用して，学習コーパス中の各単語の頻度を計測しています。
<!-- Byte-Pair Encoding (BPE) was introduced in [Neural Machine Translation of Rare Words with Subword Units (Sennrich et al., 2015)](https://arxiv.org/abs/1508.07909). 
BPE relies on a pre-tokenizer that splits the training data into words. 
Pretokenization can be as simple as space tokenization, e.g. [GPT-2](https://huggingface.co/docs/transformers/master/en/model_doc/gpt2), [Roberta](https://huggingface.co/docs/transformers/master/en/model_doc/roberta). More advanced pre-tokenization include rule-based tokenization, e.g. [XLM](https://huggingface.co/docs/transformers/master/en/model_doc/xlm),
[FlauBERT](https://huggingface.co/docs/transformers/master/en/model_doc/flaubert) which uses Moses for most languages, or [GPT](https://huggingface.co/docs/transformers/master/en/model_doc/gpt) which uses Spacy and ftfy, to count the frequency of each word in the training corpus.
-->
 
事前トークン化処理後，ユニークな単語の集合を作成し，各単語の学習データにおける出現頻度を決定します。
次に，固有単語集合に出現する全ての記号からなる基本語彙を作成し，基本語彙の 2 つの記号から新しい記号を形成するマージ規則を学習します。
これは，語彙が所望の語彙サイズに達するまで行われます。
希望する語彙サイズは，トークン化器を学習する前に定義するハイパーパラメータであることに注意してください。
<!-- After pre-tokenization, a set of unique words has been created and the frequency of each word it occurred in the training data has been determined. 
Next, BPE creates a base vocabulary consisting of all symbols that occur in the set of unique words and learns merge rules to form a new symbol from two symbols of the base vocabulary. 
It does so until the vocabulary has attained the desired vocabulary size. Note that the desired vocabulary size is a hyperparameter to define before training the tokenizer. -->

例として，事前トークン化後に，以下のような単語とその頻度のセットが決定されたとします：
<!-- As an example, let's assume that after pre-tokenization, the following set of words including their frequency has been determined: -->

```
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
```

したがって，基本語彙は  `["b", "g", "h", "n", "p", "s", "u”]`  となります。
すべての単語を基本語彙の記号に分割すると，以下のようになります。
<!-- Consequently, the base vocabulary is `["b", "g", "h", "n", "p", "s", "u"]`. 
Splitting all words into symbols of the base vocabulary, we obtain: -->

```
("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)
```

BPE は，可能性のある記号対の頻度を数え，最も頻繁に出現する記号対を選び出します。
上例では，`"h”` の後に `"u”` が続くのは  10 + 5 = 15  回です (`"hug"`  が 10  回出現する中で 10 回,
`"hugs"` が  5  回出現する中で 5 回)。 
しかし，最も頻出する記号対は `"u"` の次に `"g"` で，合計で  $10 + 5 + 5 = 20$  回出現する。
したがって，トークン化器が最初に学習する結合則は，`"u"` の後に `"g"` が続く記号をすべて一緒にすることです。
次に，`"ug”` が語彙に追加されます。
その結果，単語の集合は次のようになります。
<!-- BPE then counts the frequency of each possible symbol pair and picks the symbol pair that occurs most frequently. 
In the example above `"h"` followed by `"u"` is present _10 + 5 = 15_ times (10 times in the 10 occurrences of `"hug"`, 5 times in the 5 occurrences of `"hugs"`). 
However, the most frequent symbol pair is `"u"` followed by `"g"`, occurring _10 + 5 + 5 = 20_ times in total. 
Thus, the first merge rule the tokenizer learns is to group all `"u"` symbols followed by a `"g"` symbol together. 
Next, `"ug"` is added to the vocabulary. The set of words then becomes -->

```
("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)
```

次に BPE は，最も一般的な記号対を特定します。
それは `"u"` に続いて `"n"` で 16 回出現しています。
`"u”`, "n”` は `"un"` に統合され，語彙に追加されます。
次に頻出するのは `"h"`  に続く `"ug"` で 15 回出現しています。
ここでも対は結合され `"hug”` が語彙に追加されます。
<!-- BPE then identifies the next most common symbol pair. 
It's `"u"` followed by `"n"`, which occurs 16 times. 
`"u"`, `"n"` is merged to `"un"` and added to the vocabulary. 
The next most frequent symbol pair is `"h"` followed by `"ug"`, occurring 15 times. 
Again the pair is merged and `"hug"` can be added to the vocabulary. -->

この段階で，語彙は `["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]` となり，固有語の集合は次のように表されます：
<!-- 
At this stage, the vocabulary is `["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]` and our set of unique words is represented as
-->

```
("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)
```

この時点でバイト対符号化  (BPE) の学習が終了したと仮定すると，学習済結合規則を新しい単語に適用します (ただし，新しい単語に基本語彙にない記号が含まれない限り)。
例えば `"bug"` という単語は  `["b", "ug"]`  にトークン化されますが， `"mug"` という単語は  `["<unk>", "ug"]` になります。
これは `"m"`  という記号が基本語彙にないことが理由です。
一般に `"m”` のような一文字は，学習データに各文字が少なくとも 1 回は含まれているため，`"<unk>"`というシンボルに置き換えられることはありません。
ですが，絵文字のような非常に特殊な文字の場合は起こりやすいようです。
<!-- Assuming, that the Byte-Pair Encoding training would stop at this point, the learned merge rules would then be applied to new words (as long as those new words do not include symbols that were not in the base vocabulary). 
For instance, the word `"bug"` would be tokenized to `["b", "ug"]` but `"mug"` would be tokenized as `["<unk>", "ug"]` since the symbol `"m"` is not in the base vocabulary. 
In general, single letters such as `"m"` are not replaced by the `"<unk>"` symbol because the training data usually includes at least one occurrence of each letter, but it is likely to happen for very special characters like emojis.
 -->

<!-- As mentioned earlier, the vocabulary size, *i.e.* the base vocabulary size + the number of merges, is a hyperparameter to choose. 
For instance [GPT](https://huggingface.co/docs/transformers/master/en/model_doc/gpt) has a vocabulary size of 40,478 since they have 478 base characters and chose to stop training after 40,000 merges. -->

### バイトレベル BPE <!-- Byte-level BPE -->

すべての Unicode 文字を基底文字とみなすと，すべての可能な基底文字を含む基底語彙は非常に大きくなります。
[GPT-2](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) では，より良い基本語彙を持つために，バイトを基本語彙とすることで，すべての基本文字が語彙に含まれることを保証しつつ，基本語彙のサイズを強制的に 256 にする巧妙なトリックが施されています。
句読点を扱うルールをいくつか追加すれば，GPT2 のトークン化器 は <unk> 記号を必要とせずにすべてのテキストをトークン化することができます。
[GPT-2](https://huggingface.co/docs/transformers/master/en/model_doc/gpt) は語彙サイズ 50,257 を持ちます。
これは 256 バイトの基本トークン，特別なテキスト終了トークン，および 50,000 回のマージで学習した記号に対応するものです。
<!-- A base vocabulary that includes all possible base characters can be quite large if *e.g.* all unicode characters are considered as base characters. 
To have a better base vocabulary, [GPT-2](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) uses bytes
as the base vocabulary, which is a clever trick to force the base vocabulary to be of size 256 while ensuring that every base character is included in the vocabulary. 
With some additional rules to deal with punctuation, the GPT2's tokenizer can tokenize every text without the need for the <unk> symbol. 
[GPT-2](https://huggingface.co/docs/transformers/master/en/model_doc/gpt) has a vocabulary size of 50,257, which corresponds to the 256 bytes base tokens, a special end-of-text token and the symbols learned with 50,000 merges.
-->

<a id='wordpiece'></a>

#### ワードピース WordPiece

WordPiece は [BERT](https://huggingface.co/docs/transformers/master/en/model_doc/bert), [DistilBERT](https://huggingface.co/docs/transformers/master/en/model_doc/distilbert), [Electra](https://huggingface.co/docs/transformers/master/en/model_doc/electra) で使用されたサブワード (部分単語) トークン化アルゴリズムです。
このアルゴリズムは [Japanese and Korean Voice Search (Schuster et al., 2012)](https://static.googleusercontent.com/media/research.google.com/ja//pubs/archive/37842.pdf) で概説されており，BPE と非常に類似しています。
WordPiece はまず，学習データに含まれるすべての文字を含むように語彙を初期化し，所定の数の統合規則を漸進的に学習する。
BPE とは対照的に，WordPiece は最も頻度の高い記号対を選ぶのではなく，一度語彙に追加された学習データの尤度を最大化するものを選びます。
<!-- WordPiece is the subword tokenization algorithm used for [BERT](https://huggingface.co/docs/transformers/master/en/model_doc/bert), [DistilBERT](https://huggingface.co/docs/transformers/master/en/model_doc/distilbert), and [Electra](https://huggingface.co/docs/transformers/master/en/model_doc/electra). 
The algorithm was outlined in [Japanese and Korean Voice Search (Schuster et al., 2012)](https://static.googleusercontent.com/media/research.google.com/ja//pubs/archive/37842.pdf) and is very similar to BPE. 
WordPiece first initializes the vocabulary to include every character present in the training data and progressively learns a given number of merge rules. 
In contrast to BPE, WordPiece does not choose the most frequent symbol pair, but the one that maximizes the likelihood of the training data once added to the vocabulary.-->

では，これは具体的にどのような意味でしょうか？
先ほどの例で言えば，学習データの尤度を最大にするということは，、最初のシンボルと 2 番目のシンボルの確率を割った値が，全ての記号対の中で最大となる記号対を見つけることに相当します。
*例：`"u”`,  `"g”` の順は, `"ug”` を `"u”`, `"g”` で割った確率が，他のどの記号ペアよりも大きい場合にのみ結合されたことになります。
直感的には，WordPiece は BPE とは少し異なり，2 つのシンボルをマージすることによって失うものを評価し，それが「価値あるもの」であることを確認するのです。
<!-- So what does this mean exactly? 
Referring to the previous example, maximizing the likelihood of the training data is equivalent to finding the symbol pair, whose probability divided by the probabilities of its first symbol followed by its second symbol is the greatest among all symbol pairs. 
*E.g.* `"u"`, followed by `"g"` would have only been merged if the probability of `"ug"` divided by `"u"`, `"g"` would have been greater than for any other symbol pair. 
Intuitively, WordPiece is slightly different to BPE in that it evaluates what it _loses_ by merging two symbols to make ensure it's _worth it_.
-->

<a id='unigram'></a>

#### ユニグラム Unigram

Unigram は，[Subword Regularization] で紹介されたサブワード (部分単語)  トークン化アルゴリズムです。
[Improving Neural Network Translation Models with Multiple Subword Candidates (Kudo, 2018)】(https://arxiv.org/pdf/1804.10959.pdf) で紹介されています。
BPE や WordPiece とは対照的に， Unigram はベース語彙を大量の記号に初期化し，各記号を徐々に切り詰めてより小さな語彙を得る。
例えば，基本語彙はすべてのトークン化された単語と最も一般的な部分文字列に対応することができます。
Unigram は変換器のどのモデルにも直接使用されませんが [SentencePiece](#sentencepiece) と連携して使用されます。
<!-- Unigram is a subword tokenization algorithm introduced in [Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates (Kudo, 2018)](https://arxiv.org/pdf/1804.10959.pdf). 
In contrast to BPE or WordPiece, Unigram initializes its base vocabulary to a large number of symbols and progressively trims down each symbol to obtain a smaller vocabulary. 
The base vocabulary could for instance correspond to all pre-tokenized words and the most common substrings. 
Unigram is not used directly for any of the models in the transformers, but it's used in conjunction with [SentencePiece](#sentencepiece). -->

各訓練ステップにおいて，Unigram アルゴリズムは現在の語彙と Unigram 言語モデルが与えられた訓練データに対する損失 （しばしば対数尤度として定義されます) を定義します。
そして，語彙内の各シンボルについて，そのシンボルが語彙から削除された場合に全体の損失がどの程度増加するかを計算する。
そして，ユニグラム は損失の増加が最も少ない p 個 ( p は通常 10％ または 20％) の記号，すなわち学習データに対する全体の損失に最も影響を与えない記号を削除します。
この処理を語彙が目的のサイズになるまで繰り返す。
Unigram アルゴリズムは常に基本文字を保持するため，どのような単語もトークン化することができます。
<!-- At each training step, the Unigram algorithm defines a loss (often defined as the log-likelihood) over the training data given the current vocabulary and a unigram language model. 
Then, for each symbol in the vocabulary, the algorithm computes how much the overall loss would increase if the symbol was to be removed from the vocabulary. Unigram then removes p (with p usually being 10% or 20%) percent of the symbols whose loss increase is the lowest, *i.e.* those symbols that least affect the overall loss over the training data. 
This process is repeated until the vocabulary has reached the desired size. 
The Unigram algorithm always keeps the base characters so that any word can be tokenized. -->

Unigram はマージルールに基づいていないため (BPE や WordPiece とは対照的))， アルゴリズムは学習後の新しいテキストをトークン化する方法をいくつか持っています。
たとえば，訓練された  Unigram トークン化器が語彙を表示する場合，次のようになります。
<!--
Because Unigram is not based on merge rules (in contrast to BPE and WordPiece), the algorithm has several ways of tokenizing new text after training. 
As an example, if a trained Unigram tokenizer exhibits the vocabulary:
-->

```
["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"],
```

`"hugs”` は `["hug", "s”]`,  `["h", "ug", "s”]`,  `["h", "u", "g", "s”]` のいずれにもトークン化できます。
では，どれを選べばいいのでしょうか？
Unigram は，学習後に各可能なトークン化の確率を計算できるように，語彙を保存するだけでなく，学習コーパスの各トークンの確率を保存します。
このアルゴリズムでは，実際に最も可能性の高いトークンを選択しますが，確率に応じて可能性のあるトークンをサンプリングすることもできます。
<!-- 
`"hugs"` could be tokenized both as `["hug", "s"]`, `["h", "ug", "s"]` or `["h", "u", "g", "s"]`. 
So which one to choose? 
Unigram saves the probability of each token in the training corpus on top of saving the vocabulary so that the probability of each possible tokenization can be computed after training. 
The algorithm simply picks the most likely tokenization in practice, but also offers the possibility to sample a possible tokenization according to their probabilities. -->

これらの確率は，トークン化器が学習される損失によって定義されます。
学習データが $x_{1}, \dots, x_{N}$ という単語で構成されていて，ある単語 $x_{i}$ に対して可能なすべてのトークン化の集合を $S(x_{i})$ と定義すると，全体の損失は次のように定義されます。
<!-- Those probabilities are defined by the loss the tokenizer is trained on. 
Assuming that the training data consists of the words $x_{1}, \dots, x_{N}$ and that the set of all possible tokenizations for a word $x_{i}$ is defined as $S(x_{i})$, then the overall loss is defined as -->

$$
\mathcal{L} = -\sum_{i=1}^{N} \log \left(\sum_{x \in S(x_{i})} p(x) \right)
$$

<a id='sentencepiece'></a>

#### SentencePiece

これまで説明してきたトークン化アルゴリズムには同じ問題があります。
それは，入力テキストでは単語の区切りに空白が使われていると想定されていることです。
しかし，すべての言語で単語の区切りにスペースが使われているわけではありません。
この問題を解決するためには，言語固有の事前トークン化器を使用することが考えられます。
例えば、 [XLM](https://huggingface.co/docs/transformers/master/en/model_doc/xlm) では，中国語，日本語，タイ語の事前トークン化器を使用しています)。
この問題をより一般的に解決するために [SentencePiece: A simple and language independent subword tokenizer and detokenizer for Neural Text Processing (Kudo et al., 2018)](https://arxiv.org/pdf/1808.06226.pdf) では，入力を生の入力ストリームとして扱うため，使用する文字のセットにスペースが含まれます。
そして，BPE または unigram アルゴリズムを使用して，適切な語彙を構築します。
<!-- All tokenization algorithms described so far have the same problem: It is assumed that the input text uses spaces to separate words. 
However, not all languages use spaces to separate words. 
One possible solution is to use language specific pre-tokenizers, *e.g.* [XLM](https://huggingface.co/docs/transformers/master/en/model_doc/xlm) uses a specific Chinese, Japanese, and Thai pre-tokenizer). 
To solve this problem more generally, [SentencePiece: A simple and language independent subword tokenizer and detokenizer for Neural Text Processing (Kudo et al., 2018)](https://arxiv.org/pdf/1808.06226.pdf) treats the input as a raw input stream, thus including the space in the set of characters to use. 
It then uses the BPE or unigram algorithm to construct the appropriate vocabulary.-->

[XLNetTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/xlnet#transformers.XLNetTokenizer) では，例えば SentencePiece を使用しており，先ほどの例で `"▁”` の文字が語彙に含まれていたのもそのためです。
SentencePieceを 使った復号化は，すべてのトークンを連結し，`"▁”` をスペースに置き換えるだけなので，とても簡単です。
<!--
The [XLNetTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/xlnet#transformers.XLNetTokenizer) uses SentencePiece for example, which is also why in the example earlier the `"▁"` character was included in the vocabulary. 
Decoding with SentencePiece is very easy since all tokens can just be concatenated and `"▁"` is replaced by a space.-->

SentencePieceを使用するライブラリのすべての変換モデルは unigram と組み合わせて使用しています。
SentencePiece を使用するモデルの例としては，[ALBERT](https://huggingface.co/docs/transformers/master/en/model_doc/albert), [XLNet](https://huggingface.co/docs/transformers/master/en/model_doc/xlnet), [Marian](https://huggingface.co/docs/transformers/master/en/model_doc/marian), [T5](https://huggingface.co/docs/transformers/master/en/model_doc/t5) があります。
<!--
All transformers models in the library that use SentencePiece use it in combination with unigram. 
Examples of models using SentencePiece are [ALBERT](https://huggingface.co/docs/transformers/master/en/model_doc/albert), [XLNet](https://huggingface.co/docs/transformers/master/en/model_doc/xlnet), [Marian](https://huggingface.co/docs/transformers/master/en/model_doc/marian), and [T5](https://huggingface.co/docs/transformers/master/en/model_doc/t5). -->